# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
import pandas as pd
import xlwings as xw
import csv

In [ ]:
reviews = pd.read_csv('data/reviews_sample.csv', index_col=0)
recipes = pd.read_csv('data/recipes_sample.csv', usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])

In [ ]:
reviews.sample(5)

In [ ]:
recipes.sample(5)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [ ]:
recipe_sample = recipes.sample(frac = 0.05)
review_sample = reviews.sample(frac = 0.05)

with pd.ExcelWriter('recipes.xlsx') as writer:
    recipe_sample.to_excel(writer, sheet_name='Рецепты')
    review_sample.to_excel(writer, sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
wb = xw.Book('recipes.xlsx')
recipe_sheet = wb.sheets['Рецепты']

recipe_sheet.range('H1').options(index=False).value = (recipe_sample["minutes"] * 60).rename("seconds_assign")

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
recipe_sheet.range('I1').value = 'seconds_formula'
row = len(recipe_sample)

recipe_sheet.range(f'I2:I{row}').formula = '=D2*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
recipe_sheet.range('H1:I1').api.Font.Bold = True
recipe_sheet.range('H1:I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for current_row in range(2, row + 1):
    time = recipe_sheet.range(f'D{current_row}').value

    if time < 5:
        recipe_sheet.range(f'D{current_row}').color = (127, 255, 0)
    elif time <= 10:
        recipe_sheet.range(f'D{current_row}').color = (255, 215, 0)
    else:
        recipe_sheet.range(f'D{current_row}').color = (238, 44, 44)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
recipe_sheet.range('J1').value = 'n_reviews'
recipe_sheet.range(f'J2:J{row}').formula = f'=COUNTIF(Отзывы!C:C, $C2)'

In [ ]:
wb.save()
wb.close()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    review_sheet = wb.sheets['Отзывы']
    review_max_row = len(review_sample)

    for i in range(2, review_max_row + 1):
        rating = review_sheet.range(f'E{i}').value
        id = review_sheet.range(f'C{i}').value

        if not (0 <= rating <= 5 and recipe_sheet.range('A:A').find(id) is not None):
            review_sheet[f'A{i}:F{i}'].color = (238, 44, 44)

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
with open('data/recipes_model.csv', 'r', encoding='UTF-8') as file:
    data = list(csv.reader(file, delimiter='\t'))

recipes_model_name = 'recipes_model.xlsx'

if not xw.Book(recipes_model_name).sheets:
    wb = xw.Book()
    wb.sheets.add(name='Модель')
    wb.save(recipes_model_name)

wb = xw.Book(recipes_model_name)
model_sheet = wb.sheets['Модель']

model_sheet.range('A2').value = data

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
model_sheet.range('J2').formula = f"=$B2 & " " & UPPER(C2) & IF(D2='PK', ' PRIMARY KEY', IF(D2='FK', ' REFERENCES ' & $H2 & '(' & $I2 & ')', IF(D2<>'PK' & F2='Y', ' NOT NULL', '')))"

model_max_row = last_row = model_sheet.range('B:B').end('down').row

model_sheet.range('J2').api.AutoFill(model_sheet.range(f'J2:J{model_max_row}').api, 0)

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
title_row = model_sheet.range('A1').expand('right')
title_row.color = '00ccff'
title_row.api.Font.Bold = True
model_sheet.api.Cells.EntireColumn.AutoFit()
title_row.api.AutoFilter(Field=1)

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot)

In [ ]:
wb.save()
wb.close()